# Ly-$\alpha$ Emitter Lens Candidates

Spin through a list of strong lens candidates identified during VI of Blanc spectra and plot the *cumulative* daily coadds and Legacy Survey cutouts of the targets.

In [1]:
import os
from glob import glob

from astropy.io import ascii, fits
from astropy.table import join, hstack, vstack, unique, Table
from desispec.spectra import stack as specstack
from desispec.io import read_spectra, write_spectra
from desispec.coaddition import coadd_cameras

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd

## Read in Data

Access CSV table of objects.

In [2]:
lyman_alpha_all = ascii.read('Lyman_alpha_emitter.csv', format='csv')
data = lyman_alpha_all['TARGET_RA', 'TARGET_DEC', 'TARGETID', 'TILEID']
data

TARGET_RA,TARGET_DEC,TARGETID,TILEID
float64,float64,int64,int64
36.0399307570936,-4.934008419977485,39627664624982144,80606
37.61098456933181,-5.5721365281361095,39627652612496940,80606
36.897979060591005,-4.9944037902580245,39627664641754681,80606
35.794242561958725,-4.723924304598347,39627670643808885,80606
36.31708666799188,-3.5771263201986288,39627700796654265,80606
36.693310096235535,-5.1686833422902865,39627658614541257,80606
36.717037481403295,-5.23623388409939,39627658614542070,80606
36.84097021155235,-5.873744894748536,39627646585280781,80606
36.50963889004555,-3.552235986889796,39627700796661013,80606


## Access Reduced Spectra

Find the spectroscopic reductions of these candidates. Then find the corresponding redshift fits and store those with the spectra in the `extra_catalog` table in the `Spectra` class.

To change the reduction, just change the `redux` variable below. Possible options are `daily/tiles/cumulative`, `blanc/tiles`, `cascades/tiles`, `denali/tiles`, etc.

In [3]:
# Change reduction: daily, denali, etc.
redux = 'daily/tiles/cumulative'
tobj=[] # Stores objects that are found
fobj=[] # Stores objects that are not found anywhere

# Loop through tile list and extract spectra.
lenspec = None
tiles = np.unique(data['TILEID'])

for tile in tiles:
    print(tile)
    tile_folders = sorted(glob('{}/{}/{}/*'.format(os.environ['DESI_SPECTRO_REDUX'], redux, tile)))
    if tile_folders:
        # Grab coadds from the last available folder.
        tile_folder = tile_folders[-1]
        coadds = sorted(glob('{}/coadd*'.format(tile_folder)))
        
        # Coadds are organized by petal ID. Search each file for matching targets.
        for coadd in coadds:
            try:
                spectra = read_spectra(coadd)
                
                # Match TARGETID.
                sselect = np.in1d(spectra.fibermap['TARGETID'], data['TARGETID'])
                if np.any(sselect):
                    spectra = spectra[sselect]
                    if not hasattr(spectra, 'scores_comments'):
                        spectra.scores_comments = None
                    
                    # Extract corresponding best-fit redshift.
                    zbfile = coadd.replace('coadd', 'zbest')
                    zbest = fits.open(zbfile)['ZBEST'].data
                    zselect = np.in1d(zbest['TARGETID'], data['TARGETID'])
                    zbest = zbest[zselect]

                    # Append spectra to a larger list of spectra, stored in memory.
                    # Note that an EXPID is required in the update step below, but the coadded spectra
                    # have a FIRST_EXPID and a LAST_EXPID. So copy one of these as a hack.
                    spectra.fibermap['EXPID'] = spectra.fibermap['LAST_EXPID']
                    spectra.extra_catalog = zbest
                    
                    if lenspec is None:
                        lenspec = spectra
                    else:
                        lenspec = specstack([lenspec, spectra])
            except OSError as e:
                print(e)

if not hasattr(lenspec, 'scores_comments'):
    lenspec.scores_comments = None

80606
INFO:spectra.py:253:read_spectra: iotime 0.757 sec to read coadd-0-80606-thru20201219.fits at 2021-06-03T13:58:41.959513
Header missing END card.
INFO:spectra.py:253:read_spectra: iotime 0.802 sec to read coadd-1-80606-thru20201219.fits at 2021-06-03T13:58:43.481723
Header missing END card.
INFO:spectra.py:253:read_spectra: iotime 1.007 sec to read coadd-2-80606-thru20201219.fits at 2021-06-03T13:58:45.422882
Header missing END card.
INFO:spectra.py:253:read_spectra: iotime 0.785 sec to read coadd-3-80606-thru20201219.fits at 2021-06-03T13:58:47.031097
Header missing END card.
INFO:spectra.py:253:read_spectra: iotime 0.834 sec to read coadd-4-80606-thru20201219.fits at 2021-06-03T13:58:48.959998
Header missing END card.
INFO:spectra.py:253:read_spectra: iotime 1.060 sec to read coadd-5-80606-thru20201219.fits at 2021-06-03T13:58:51.012054
Header missing END card.
INFO:spectra.py:253:read_spectra: iotime 0.759 sec to read coadd-6-80606-thru20201219.fits at 2021-06-03T13:58:52.5584

In [4]:
lenspec.num_spectra(), lenspec.num_targets(), len(np.unique(data['TARGETID'])), len(data)

(64, 64, 64, 64)

## Plot Spectra and Cutouts

Loop through all identified spectra and redshift fits and plot them along with the image cutouts.

In [5]:
import requests

def get_cutout(targetid, ra, dec, verbose=False):
    """Grab and cache legacy survey cutouts.
    
    Parameters
    ----------
    targetid : int
        DESI target ID.
    ra : float
        Right ascension (degrees).
    dec : float
        Declination (degrees).
    verbose : bool
        Add some status messages if true.
        
    Returns
    -------
    img_name : str
        Name of JPG cutout file written after query.
    """
    img_name = '{}.jpg'.format(targetid)
    
    if os.path.exists(img_name):
        if verbose:
            print('{} exists.'.format(img_name))
    else:
        if verbose:
            print('Accessing {}'.format(img_name))
        img_url = 'https://www.legacysurvey.org/viewer/cutout.jpg?ra={}&dec={}&%22/pix=0.25&layer=dr8&size=180'.format(ra, dec)
        with open(img_name, 'wb') as handle: 
            response = requests.get(img_url, stream=True) 
            if not response.ok: 
                print(response) 
            for block in response.iter_content(1024): 
                if not block: 
                    break 
                handle.write(block)
    
    return img_name


In [6]:
from scipy.ndimage import gaussian_filter1d

mpl.rc('figure', max_open_warning = 0)

In [7]:
# Emission and absorption lines from Prospect tables.
emi_lines = ascii.read('emission_lines.txt', comment='#', names=['name','longname','lambda','vacuum','major'])
abs_lines = ascii.read('absorption_lines.txt', comment='#', names=['name','longname','lambda','vacuum','major'])

emi_lines_major = emi_lines[emi_lines['major']=='True']
abs_lines_major = abs_lines[abs_lines['major']=='True']

### Coadd Spectra

Coadd spectra from the different spectrograph arms.

In [8]:
coadd_spec = coadd_cameras(lenspec)

### Make Plots

In [9]:
for i in range(lenspec.num_spectra()):
    
    fig, axes = plt.subplots(1,2, figsize=(16,5), gridspec_kw={'width_ratios':[3,1.1]},
                             tight_layout=True)
    
    # Plot the spectra
    ax = axes[0]
    
    # Plot the spectra from each spectrograph.
    for band in 'brz':
        smoothed = gaussian_filter1d(lenspec.flux[band][i], 5)
        ax.plot(lenspec.wave[band], smoothed, alpha=0.5, label='{} camera'.format(band))
        
    # Coadd the cameras and plot the resulting flux.
    smoothed = gaussian_filter1d(coadd_spec.flux['brz'][i], 5)
    fmin = 0.9*np.min(smoothed)
    fmax = 1.1*np.max(smoothed)
    if fmin == fmax:
        fmin, fmax = -1, 1
    ax.plot(coadd_spec.wave['brz'], smoothed, color='gray', label='coadd')
        
    zbest = lenspec.extra_catalog[i]
    z = zbest['Z']
    dchi2 = zbest['DELTACHI2']
    zwarn = zbest['ZWARN']
    sptype = zbest['SPECTYPE']
    
    for eline in emi_lines:
        wl = eline['lambda']*(1 + z)
        if wl > 3600 and wl < 9800:
            ax.axvline(wl, ls='--', color='k', alpha=0.3)
            ax.text(wl+20, fmin, eline['name'], fontsize=8, rotation=90, alpha=0.3)

    for aline in abs_lines:
        wl = aline['lambda']*(1 + z)
        if wl > 3600 and wl < 9800:
            ax.axvline(wl, ls='--', color='r', alpha=0.3)
            ax.text(wl+20, 0.95*fmax, aline['name'], color='r', fontsize=8, rotation=90, alpha=0.3)

    ax.set(xlabel=r'$\lambda_{\mathrm{obs}}$ [$\AA$]',
           xlim=(3500,9900),
           ylabel=r'flux [erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$]',
           ylim=(fmin, fmax),
           title=r'{}; $z={:.3f}$ ($\Delta\chi^2={:.5g}$; ZWARN=0x{:x}; SPECTYPE={:s})'.format(lenspec.fibermap[i]['TARGETID'], z, dchi2, zwarn, sptype.decode('utf-8')),
          )
    
    ax.legend(loc='lower right', fontsize=8)
    
    # Plot the image cutout.
    ax = axes[1]
    
    obj = lenspec.fibermap[i]
    img_file = get_cutout(obj['TARGETID'], obj['TARGET_RA'], obj['TARGET_DEC'])
    img = mpl.image.imread(img_file)

    ax.imshow(img)

    x1, x2, x3, x4 = [90, 90], [70, 80], [90, 90], [100,110]
    y1, y2, y3, y4 = [70, 80], [90, 90], [100,110], [90,90]
    ax.plot(x1, y1, x2, y2, x3, y3, x4, y4, color='r', linewidth=2, alpha=0.7)
    ax.text(5,15, '{:3s} = {}\n{:3s} = {}'.format('RA', obj['TARGET_RA'], 'Dec', obj['TARGET_DEC']), color='yellow', fontsize=9)
    ax.set(aspect='equal',
           title='{}, Tile {}, Exp {}'.format(obj['LAST_NIGHT'], obj['TILEID'], obj['EXPID']))
    ax.axis('off')
    
    print(obj['TARGETID'], obj['EXPID'], obj['TILEID'], obj['FIBER'])
    
    fig.savefig('lens_{}_{}_{:06d}_{:06d}.png'.format(obj['TARGETID'], obj['LAST_NIGHT'], obj['TILEID'], obj['EXPID']), dpi=100)
    
    # Clean up to avoid memory problems.
    fig.clear()
    plt.close(fig)

39627646585280781 68813 80606 330
39627652599907489 68813 80606 360
39627664633371817 68813 80606 125
39627640553871919 68813 80606 508
39627664624976530 68813 80606 943
39627664624982144 68813 80606 710
39627658589374759 68813 80606 1375
39627670635422855 68813 80606 1305
39627670643808885 68813 80606 1290
39627688712864357 68813 80606 1953
39627688717058772 68813 80606 1668
39627688729645736 68813 80606 2275
39627694765251168 68813 80606 2519
39627700796654265 68813 80606 2982
39627700796661013 68813 80606 2704
39627706823875576 68813 80606 2594
39627682702435003 68813 80606 3443
39627700805044666 68813 80606 3035
39627682727592135 68813 80606 3596
39627694782027010 68813 80606 3820
39627658631318160 68813 80606 4165
39627658631318944 68813 80606 4365
39627658631321939 68813 80606 4396
39627664654336508 68813 80606 4046
39627670668970656 68813 80606 4138
39627670668973482 68813 80606 4311
39627670673166752 68813 80606 4047
39627676704571527 68813 80606 4477
39627676704574732 68813 80

In [10]:
write_spectra('lyman_alpha_emitter_desi_dailyspectra.fits', lenspec)

INFO:spectra.py:154:write_spectra: iotime 1.114 sec to write lyman_alpha_emitter_desi_dailyspectra.fits at 2021-06-03T14:00:40.627745


'/global/u2/s/sybenzvi/desi/stronglens/doc/nb/lyman_alpha_emitter_desi_dailyspectra.fits'